In [62]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor

In [43]:
column_names = pd.read_csv("hw3-data/my_train.csv", nrows=1).columns.tolist()

train_data = pd.read_csv("hw3-data/my_train.csv", sep=",", skiprows=1, names=column_names, engine="python")
dev_data = pd.read_csv("hw3-data/my_dev.csv", sep=",", skiprows=1, names=column_names, engine="python")
test_data = pd.read_csv("hw3-data/test.csv", sep=",", skiprows=1, names=column_names[:-1], engine="python")

# Separate features and target variable
X_train = train_data[column_names[1:-1]]  # Features
y_train = np.log(train_data['SalePrice'])  # Target variable

X_dev = dev_data[column_names[1:-1]]  # Features
y_dev = np.log(dev_data['SalePrice'])  # Target variable

# 'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 
# 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'YearBuilt', 'YearRemodAdd', 'MoSold', 'YrSold'

num_cols = [
    'LotFrontage', 'LotArea', 
    'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 
    'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
    'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 
    'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'YearBuilt', 'YearRemodAdd', 'MoSold', 'YrSold'
]

cat_cols = np.array([c for c in column_names[1:-1] if c not in num_cols])

poly_cols = [
    'LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF'
]

num_cols = np.array([c for c in num_cols if c not in poly_cols])

X_train[np.concatenate((num_cols, poly_cols))] = X_train[np.concatenate((num_cols, poly_cols))].fillna(0)
X_dev[np.concatenate((num_cols, poly_cols))] = X_dev[np.concatenate((num_cols, poly_cols))].fillna(0)

X_train[cat_cols] = X_train[cat_cols].astype(str)
X_dev[cat_cols] = X_dev[cat_cols].astype(str)

num_processor = 'passthrough'
# num_processor = MinMaxScaler()
cat_processor = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

preprocessor = ColumnTransformer([
    ('cat', cat_processor, cat_cols),
    ('num', num_processor, num_cols),
    ('poly', num_processor, poly_cols)
])

# Binarization
X_train_binary = preprocessor.fit_transform(X_train)
X_dev_binary = preprocessor.transform(X_dev)

# method 2: PolynomialFeatures
ab = PolynomialFeatures(degree=2)
X_train_poly = ab.fit_transform(X_train[poly_cols])
X_dev_poly = ab.transform(X_dev[poly_cols])

new_x_train_binary = np.concatenate((X_train_binary, X_train_poly), axis=1)
new_X_dev_binary = np.concatenate((X_dev_binary, X_dev_poly), axis=1)

# model = LinearRegression().fit(new_x_train_binary, y_train)

ridge_model = Ridge(alpha=10)
ridge_model.fit(new_x_train_binary, y_train)

y_pred = ridge_model.predict(new_X_dev_binary)

# Make predictions
# y_pred = model.predict(new_X_dev_binary)

np.sqrt(mean_squared_error(y_dev, y_pred))

/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_14783/4284695999.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[np.concatenate((num_cols, poly_cols))] = X_train[np.concatenate((num_cols, poly_cols))].fillna(0)
/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_14783/4284695999.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_dev[np.concatenate((num_cols, poly_cols))] = X_dev[np.concatenate((num_cols, poly_cols))].fillna(0)
/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn

0.11893659362776476

In [44]:
X_test = test_data[column_names[1:-1]]  # Features

X_test[np.concatenate((num_cols, poly_cols))] = X_test[np.concatenate((num_cols, poly_cols))].fillna(0)
X_test[cat_cols] = X_test[cat_cols].astype(str)

X_test_binary = preprocessor.transform(X_test)

X_test_poly = ab.transform(X_test[poly_cols])

new_X_test_binary = np.concatenate((X_test_binary, X_test_poly), axis=1)

# Make predictions
y_test_pred = np.exp(ridge_model.predict(new_X_test_binary))

# Assigning new predicted column to the dataset
test_data['SalePrice'] = y_test_pred

# Create a new DataFrame with 'ID' and 'SalePrice'
result_df = test_data[['Id', 'SalePrice']]

# Save the DataFrame to a CSV file with a header
result_df.to_csv('smart_ridge_poly.csv', index=False)

/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_14783/2690897794.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[np.concatenate((num_cols, poly_cols))] = X_test[np.concatenate((num_cols, poly_cols))].fillna(0)
/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_14783/2690897794.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[cat_cols] = X_test[cat_cols].astype(str)


### My own

In [66]:
column_names = pd.read_csv("hw3-data/my_train.csv", nrows=1).columns.tolist()

train_data = pd.read_csv("hw3-data/my_train.csv", sep=",", skiprows=1, names=column_names, engine="python")
dev_data = pd.read_csv("hw3-data/my_dev.csv", sep=",", skiprows=1, names=column_names, engine="python")
test_data = pd.read_csv("hw3-data/test.csv", sep=",", skiprows=1, names=column_names[:-1], engine="python")

# Separate features and target variable
X_train = train_data[column_names[1:-1]]  # Features
y_train = np.log(train_data['SalePrice'])  # Target variable

X_dev = dev_data[column_names[1:-1]]  # Features
y_dev = np.log(dev_data['SalePrice'])  # Target variable

# 'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 
# 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'YearBuilt', 'YearRemodAdd', 'MoSold', 'YrSold'

num_cols = [
    'LotFrontage', 'LotArea', 
    'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 
    'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
    'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 
    'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'YearBuilt', 'YearRemodAdd', 'MoSold', 'YrSold'
]

cat_cols = np.array([c for c in column_names[1:-1] if c not in num_cols])

X_train[num_cols] = X_train[num_cols].fillna(0)
X_dev[num_cols] = X_dev[num_cols].fillna(0)

X_train[cat_cols] = X_train[cat_cols].astype(str)
X_dev[cat_cols] = X_dev[cat_cols].astype(str)

num_processor = 'passthrough'
cat_processor = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

preprocessor = ColumnTransformer([
    ('cat', cat_processor, cat_cols),
    ('num', num_processor, num_cols),
])

# Binarization
X_train_binary = preprocessor.fit_transform(X_train)
X_dev_binary = preprocessor.transform(X_dev)

# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train_binary, y_train)

y_pred = rf.predict(X_dev_binary)

# Make predictions
# y_pred = model.predict(new_X_dev_binary)

np.sqrt(mean_squared_error(y_dev, y_pred))

/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_14783/23139004.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[num_cols] = X_train[num_cols].fillna(0)
/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_14783/23139004.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_dev[num_cols] = X_dev[num_cols].fillna(0)
/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_14783/23139004.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

0.1368054824037724

In [61]:
X_test = test_data[column_names[1:-1]]  # Features

X_test[num_cols] = X_test[num_cols].fillna(0)
X_test[cat_cols] = X_test[cat_cols].astype(str)

X_test_binary = preprocessor.transform(X_test)

# Make predictions
y_test_pred = np.exp(rf.predict(X_test_binary))

# Assigning new predicted column to the dataset
test_data['SalePrice'] = y_test_pred

# Create a new DataFrame with 'ID' and 'SalePrice'
result_df = test_data[['Id', 'SalePrice']]

# Save the DataFrame to a CSV file with a header
result_df.to_csv('random_forest.csv', index=False)

/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_14783/1343998965.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[num_cols] = X_test[num_cols].fillna(0)
/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_14783/1343998965.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[cat_cols] = X_test[cat_cols].astype(str)
